# Intialisation Steps
Connecting to Snowflake

## Importing required modules

In [26]:
import pandas as pd
from drg_connect import Snowflake
import qgrid 
from datetime import timedelta, datetime
import math

import warnings
warnings.filterwarnings('ignore')

## Snowflake connection parameter

In [3]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW',database='SANDBOX_ANALYTICS',schema = 'SANDBOX')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# Queries from here

## To do:
   ### patients with a dx of Eosinophilic Esophagitis (EoE)
   ### claims/EHR records (tests) before the dx of EoE
       look at allergies table in EHR? EoE is mostly diagnosed in people with protein and dairy allergies
   ### longitudinality of data

# Dx codes

In [4]:
snow.select("select * from RWD_DB.RWD.ICD_GROUPER limit 3")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,1,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A00,No map,No map,Cholera,Cholera,2017-11-02,2017-11-02
1,2,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A000,0010,Cholera due to vibrio cholerae,Cholera due to Vibrio cholerae 01 biovar cholerae,Cholera due to Vibrio cholerae 01 biovar cholerae,2017-11-02,2017-11-02
2,3,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A001,0011,Cholera due to vibrio cholerae el tor,Cholera due to Vibrio cholerae 01 biovar eltor,Cholera due to Vibrio cholerae 01 biovar eltor,2017-11-02,2017-11-02


In [6]:
%%read_sql

select
    *
from
    RWD_DB.RWD.ICD_GROUPER
where
    level_4_short_description_icd10 ilike '%eosinophilic%'
    and level_4_short_description_icd10 ilike '%esophagitis%'

Query started at 05:47:17 PM India Standard Time; Query executed in 0.05 m

,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,13184,K00 - K9589,Diseases of the digestive system,K20-K31,Diseases of esophagus stomach and duodenum,K20-K209,Esophagitis,K200,53013,Eosinophilic esophagitis,Eosinophilic esophagitis,Eosinophilic esophagitis,2017-11-02,2017-11-02


# Cohort in obs window

In [13]:
%%read_sql
create or replace table st_eoe as 

select
    patient_id,
    claim_number,
    diagnosis,
    diagnosis_sequence,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS

    where left (diagnosis,4) in ('K200') 
    and left(patient_id, 5) != 'XXX -'
    --limit 1000

Query started at 06:22:18 PM India Standard Time; Query executed in 0.36 m

,status
0,Table ST_EOE successfully created.


In [12]:
%%read_sql
select * from st_eoe limit 100

Query started at 06:11:49 PM India Standard Time; Query executed in 0.05 m

,patient_id,claim_number,diagnosis,diagnosis_sequence,year_of_service
0,410495236,alb_1794632050,K200,1,2018-11-29
1,279163740,alb_1794454067,K200,1,2018-11-28
2,18801146,alb_1794456284,K200,5,2018-10-22
3,478739300,alb_1794963052,K200,2,2018-12-03
4,71609064,alb_1794440990,K200,3,2018-12-05
5,360425285,alb_1794443154,K200,3,2018-12-07
6,322780000,alb_1794665038,K200,1,2018-12-04
7,247384812,alb_1794859859,K200,6,2018-12-06
8,292910544,alb_1794865271,K200,1,2018-12-04
9,260477470,303801000000550299,K200,0,2018-12-04


In [14]:
snow.select("select count(distinct patient_id) from st_eoe where left(patient_id, 5) != 'XXX -'")

,COUNT(DISTINCT PATIENT_ID)
0,212452


## EHR data

In [16]:

%%read_sql
create or replace table st_eoe_ehr as
select 
   genpatientid,
    icd9,
    icd10,
    recordeddttm,
    left(recordeddttm, 4) as year
    
from RWD_DB.RWD.albatross_EHR_problems
where 
   icd10 in ('K200')
or ICD9 in ('53013')

Query started at 06:31:18 PM India Standard Time; Query executed in 2.53 m

,status
0,Table ST_EOE_EHR successfully created.


In [18]:
snow.select("select count(distinct genpatientid) from st_eoe_ehr where left(genpatientid, 5) != 'XXX -'")

,COUNT(DISTINCT GENPATIENTID)
0,42750


In [19]:
%%read_sql
create or replace table st_eoe_ehr1 as
select 
   patient_id,
    icd9,
    start_date,
    stop_date
    
from RWD_DB.RWD.PELICAN_DIAGNOSIS
where ICD9 in ('53013')

Query started at 06:40:09 PM India Standard Time; Query executed in 0.11 m

,status
0,Table ST_EOE_EHR1 successfully created.


In [24]:
snow.select("select * from RWD_DB.RWD.PELICAN_DIAGNOSIS_ICD10 where left(icd10, 4) = 'K200'")

,diagnosis_id,icd10,etl_icd10


## Applying continuous coverage rules

In [25]:
%%read_sql

create or replace table st_eoe_cont_med as

select
    patient_id,
    claim_number,
    claim_type_code,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_HEADER
where
    patient_id in (select patient_id from st_eoe)
    and year_of_service between '2014-07-01' and '2018-06-30'

Query started at 01:26:05 PM India Standard Time

Exception during reset or similar
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\pool.py", line 709, in _finalize_fairy
    fairy._reset(pool)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\pool.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 457, in do_rollback
    dbapi_connection.rollback()
  File "C:\ProgramData\Anaconda3\lib\site-packages\snowflake\connector\connection.py", line 387, in rollback
    self.cursor().execute("ROLLBACK")
  File "C:\ProgramData\Anaconda3\lib\site-packages\snowflake\connector\cursor.py", line 479, in execute
    _is_put_get=_is_put_get)
  File "C:\ProgramData\Anaconda3\lib\site-packages\snowflake\connector\cursor.py", line 368, in _execute_helper
    _no_results=_no_results)
  File "C:\ProgramData\Anaconda3\lib\site-packages\snowflake\connector\connection.py", line 652, in cmd_query
    data,

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
; Query executed in 2.89 m

,status
0,Table ST_EOE_CONT_MED successfully created.


### Check duplicates

In [27]:
%%read_sql

select count(*) from st_eoe_cont_med

Query started at 01:52:58 PM India Standard Time; Query executed in 0.10 m

,COUNT(*)
0,8563233


In [28]:
%%read_sql

with t1 as (select distinct * from st_eoe_cont_med)
select count(*) from t1

Query started at 02:35:49 PM India Standard Time; Query executed in 0.11 m

,COUNT(*)
0,8542219


In [29]:
%%read_sql

--drop table st_scz_cont_med_unique;
create or replace table st_eoe_cont_med_unique as
select distinct * from st_eoe_cont_med;
drop table st_eoe_cont_med;

Query started at 02:38:09 PM India Standard Time; Query executed in 0.09 mQuery started at 02:38:14 PM India Standard Time; Query executed in 0.04 m

,status
0,ST_EOE_CONT_MED successfully dropped.


In [31]:
%%read_sql

create or replace table st_eoe_index as

select
    patient_id,
    min(year_of_service) as index_date
from
    st_eoe
where
    left(patient_id, 5) != 'XXX -'
group by 1

Query started at 02:45:05 PM India Standard Time; Query executed in 0.10 m

,status
0,Table ST_EOE_INDEX successfully created.


In [32]:
snow.drop_table("st_eoe_cont_med_tally")

DROP TABLE IF EXISTS sandbox_analytics.sandbox.st_eoe_cont_med_tally;
Table sandbox.st_eoe_cont_med_tally dropped!


In [33]:
%%read_sql

create or replace table st_eoe_cont_med_tally as

select
    a.patient_id,
    sum(case
           when datediff(d, a.index_date, b.year_of_service) between -365 and -1
           then 1
           else 0
           end) as claim_tally_pre,
    sum(case
           when datediff(d, a.index_date, b.year_of_service) between 0 and 365
           then 1
           else 0
           end) as claim_tally_post
from
    st_eoe_index a
        join st_eoe_cont_med_unique b
            on a.patient_id = b.patient_id
group by 1

Query started at 02:45:37 PM India Standard Time; Query executed in 0.11 m

,status
0,Table ST_EOE_CONT_MED_TALLY successfully created.


### Final count

In [34]:
a = snow.select("select count(distinct patient_id) from st_eoe_cont_med_tally")
b = snow.select("select count(distinct patient_id) from st_eoe_cont_med_tally where claim_tally_pre > 0 and claim_tally_post > 0")
c = snow.select("select count(distinct patient_id) from st_eoe_cont_med_tally where claim_tally_pre > 1 and claim_tally_post > 1")
d = snow.select("select count(distinct patient_id) from st_eoe_cont_med_tally where claim_tally_pre > 2 and claim_tally_post > 2")
e = snow.select("select count(distinct patient_id) from st_eoe_cont_med_tally where claim_tally_pre > 3 and claim_tally_post > 3")
f = snow.select("select count(distinct patient_id) from st_eoe_cont_med_tally where claim_tally_pre > 4 and claim_tally_post > 4")

lst = ['pts_obs_window',
       '1_claim_pre_and_post',
       '2_claims_pre_and_post',
       '3_claims_pre_and_post',
       '4_claims_pre_and_post',
       '5_claims_pre_and_post',]

pd.concat([a,b,c,d,e,f], keys=lst)

,,COUNT(DISTINCT PATIENT_ID)
pts_obs_window,0,209704
1_claim_pre_and_post,0,157403
2_claims_pre_and_post,0,129233
3_claims_pre_and_post,0,108298
4_claims_pre_and_post,0,92294
5_claims_pre_and_post,0,79178


In [35]:
%%read_sql

create or replace table st_eoe_cohort as

select
    patient_id,
    index_date
from 
    st_eoe_index
where
    patient_id in (select patient_id 
                   from st_eoe_cont_med_tally 
                   where claim_tally_pre > 1
                   and claim_tally_pre > 1)

Query started at 02:50:12 PM India Standard Time; Query executed in 0.10 m

,status
0,Table ST_EOE_COHORT successfully created.


In [36]:
snow.select("select count(distinct patient_id) from st_eoe_cohort")

,COUNT(DISTINCT PATIENT_ID)
0,155145


## Using Pelican: diagnosis and allergy table?

In [37]:
snow.select ("select * from RWD_DB.RWD.PELICAN_diagnosis_ICD10 where icd10 in ('K200')")

,diagnosis_id,icd10,etl_icd10


In [38]:
snow.select ("select * from RWD_DB.RWD.PELICAN_diagnosis where icd9 in ('53013')")

,diagnosis_id,patient_id,provider_id,icd9,is_active,start_date,start_date_source,stop_date,last_modified,created_at,etl_icd9


In [39]:
snow.select ("select * from RWD_DB.RWD.PELICAN_allergen limit 3")

,allergen_id,description
0,225,Red kidney beans
1,226,Red Meat
2,223,Poultry


In [41]:
snow.select ("select * from RWD_DB.RWD.PELICAN_allergy where allergen_id in ('225') limit 3")

,allergy_id,allergen_id,patient_id,medication_id,start_date,stop_date,created_at,is_active
0,7683402345780138682,225,E0E43A78-5CA0-A1C1-B714-FBF1682E6E3F,None,None,None,2013-11-27,True
1,9062235091800355357,225,0982702A-8967-6FC0-18DD-7C6219DDBE22,None,None,None,2018-07-12,True
2,7806241368339597651,225,41252FA9-32BC-8153-4C2D-FB2F6C61DAAD,None,None,None,2015-09-06,True


In [47]:
%%read_sql

create or replace table st_eoe_ehr_allergy as

select allergen_id,
    created_at

from 
    RWD_DB.RWD.PELICAN_allergy
where
    patient_id in (select patient_id from st_eoe_cohort)

Query started at 03:24:56 PM India Standard Time

ProgrammingError: (snowflake.connector.errors.ProgrammingError) 100038 (22018): Numeric value 'C07FD888-1A0E-E2AD-898B-D23801187B1F' is not recognized [SQL: 'create or replace table st_eoe_ehr_allergy as\n\nselect allergen_id,\n    created_at\n\nfrom \n    RWD_DB.RWD.PELICAN_allergy\nwhere\n    patient_id in (select patient_id\n                   from st_eoe_cohort)'] (Background on this error at: http://sqlalche.me/e/f405)